<a href="https://colab.research.google.com/github/Tanucode/Deep-Learning-Practice/blob/main/LDA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import string
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
df = pd.read_excel ("games.xlsx")
print (df)

             domain             app                             review_id  \
0      productivity        things-3  c9274c0a-a120-4e09-816b-7a8ba3a16634   
1      productivity      notability  e633e20a-07c1-4a5e-80b1-b104b6cf6a61   
2      productivity  microsoft-word  506230e3-cc98-4233-be40-89e52d53990c   
3      productivity        things-3  69d44a5e-218f-4f55-8a99-6cca55d43ca1   
4      productivity      notability  3d0b634c-d402-47e8-ba7d-bf6209fed826   
...             ...             ...                                   ...   
11316         games       among-us-  70d02d85-60b0-4c94-a43d-0307f089e36e   
11317         games  subway-surfers  5431fed8-7c83-4ea1-bc4e-c2282c6f1bbd   
11318         games        monopoly  5880f046-3a22-4dd9-8b01-ab6dea2bd4f6   
11319         games       among-us-  8765dd75-e764-4d94-a2ea-f23c8894908d   
11320         games        heads-up  27d1206e-e930-4944-b4cb-5d61a2e58d91   

                                sentence_id  \
0      00808934-e8b9-42fa-b3

In [3]:
df = df.drop(columns=['app','review_id','sentence_id','title','sentence', 'rating','is_opinion','category','term','from','to'])
# df_cleaned = df.dropna()
print(df)

             domain                                             review  \
0      productivity  This new version of Things has an entirely dif...   
1      productivity  I have been using this app for over 3 years no...   
2      productivity  A lot of people use word because its very stan...   
3      productivity  I was originally skeptical on paying $7 on a t...   
4      productivity  This used to be my go to app for note taking. ...   
...             ...                                                ...   
11316         games  I used to play Among Us but not really anymore...   
11317         games  I have been playing this game since the releas...   
11318         games  NEW PROBLEM: the game consistently mis moves t...   
11319         games  So this is a great game! and if your not playi...   
11320         games  As the title states- I wish I had set a dollar...   

      sentiment  
0      positive  
1           NaN  
2      negative  
3      positive  
4      negative  
...

In [4]:
df.drop_duplicates(inplace=True)

In [5]:
df.duplicated().sum()

0

In [6]:
df= df.dropna()
print(df)

             domain                                             review  \
0      productivity  This new version of Things has an entirely dif...   
2      productivity  A lot of people use word because its very stan...   
3      productivity  I was originally skeptical on paying $7 on a t...   
4      productivity  This used to be my go to app for note taking. ...   
5      productivity  I use it for years.  It’s tagging capability i...   
...             ...                                                ...   
10868         games  NEW PROBLEM: the game consistently mis moves t...   
10896         games  So amazing game I love it but It gets boring s...   
10903         games  Among us is pretty fun but the thing is I get ...   
10921         games  This game, is literally filled with memories. ...   
10925         games  this game is good in regards to characters and...   

      sentiment  
0      positive  
2      negative  
3      positive  
4      negative  
5      positive  
...

In [7]:
df.isnull().sum()

domain       0
review       0
sentiment    0
dtype: int64

In [8]:
df['sentiment'] = df['sentiment'].replace({'positive': 1,'negative': 0})
print(df)

             domain                                             review  \
0      productivity  This new version of Things has an entirely dif...   
2      productivity  A lot of people use word because its very stan...   
3      productivity  I was originally skeptical on paying $7 on a t...   
4      productivity  This used to be my go to app for note taking. ...   
5      productivity  I use it for years.  It’s tagging capability i...   
...             ...                                                ...   
10868         games  NEW PROBLEM: the game consistently mis moves t...   
10896         games  So amazing game I love it but It gets boring s...   
10903         games  Among us is pretty fun but the thing is I get ...   
10921         games  This game, is literally filled with memories. ...   
10925         games  this game is good in regards to characters and...   

       sentiment  
0              1  
2              0  
3              1  
4              0  
5              1

In [9]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [10]:
df['review'] = df['review'].apply(remove_tags)

In [11]:
df['review'] = df['review'].apply(lambda x:x.lower())
print(df)

             domain                                             review  \
0      productivity  this new version of things has an entirely dif...   
2      productivity  a lot of people use word because its very stan...   
3      productivity  i was originally skeptical on paying $7 on a t...   
4      productivity  this used to be my go to app for note taking. ...   
5      productivity  i use it for years.  it’s tagging capability i...   
...             ...                                                ...   
10868         games  new problem: the game consistently mis moves t...   
10896         games  so amazing game i love it but it gets boring s...   
10903         games  among us is pretty fun but the thing is i get ...   
10921         games  this game, is literally filled with memories. ...   
10925         games  this game is good in regards to characters and...   

       sentiment  
0              1  
2              0  
3              1  
4              0  
5              1

In [12]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [13]:
from bs4 import BeautifulSoup


def preprocess(q):

    q = str(q).lower().strip()

    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')

    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')

    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")

    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()

    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()


    return q


In [14]:
df['review'] = df['review'].apply(preprocess)

<ipython-input-13-394ff13b77d1>:163: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  q = BeautifulSoup(q)


In [15]:
df['domain'].value_counts()

productivity         2378
social networking    1974
games                 866
Name: domain, dtype: int64

In [17]:
import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

# libraries for visualization
!pip install pyLDAvis
!pip install pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 34.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicor

In [22]:
import en_core_web_sm

nlp = en_core_web_sm.load(disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']):
       output = []
       for sent in texts:
             doc = nlp(sent)
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

In [25]:
review_list=df['review'].tolist()
print(review_list[1])
tokenized_reviews = lemmatization(review_list)
print(tokenized_reviews[1])

lot people use word standard  run problem use it  every update there s problem format  everything page 2  partly page 1  part 3  partly page 2  forth  throughout 200 pages document  try call support reach someone barely speaks english  tell happens  say  so changed something document   yes  closed document yesterday  formatted correctly  updated app today  opened wrong  must middle night i switching apples pages  thanks people work really hard make work  really great effort part  it s fun issue happens macbook ipad  what s point setting something exactly want it  simple update changes everything  it s nonsense
['lot', 'people', 'word', 'standard', 'run', 'problem', 'update', 'problem', 'format', 'page', 'page', 'part', 'page', 'page', 'document', 'support', 'tell', 'document', 'document', 'yesterday', 'app', 'today', 'wrong', 'middle', 'night', 'apple', 'page', 'thank', 'people', 'work', 'great', 'effort', 'part', 'fun', 'issue', 'ipad', 'point', 'simple', 'update', 'nonsense']


In [26]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [29]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

#LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=3, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [30]:
lda_model.print_topics()

[(0,
  '0.054*"app" + 0.018*"note" + 0.015*"time" + 0.013*"email" + 0.013*"thing" + 0.010*"feature" + 0.009*"word" + 0.008*"use" + 0.008*"work" + 0.008*"version"'),
 (1,
  '0.044*"app" + 0.019*"phone" + 0.016*"message" + 0.016*"time" + 0.015*"people" + 0.014*"call" + 0.013*"friend" + 0.012*"update" + 0.010*"discord" + 0.010*"account"'),
 (2,
  '0.090*"game" + 0.018*"time" + 0.015*"fun" + 0.014*"thing" + 0.011*"people" + 0.011*"play" + 0.011*"player" + 0.009*"good" + 0.009*"money" + 0.009*"level"')]

In [37]:

df['lda_topics'] = df['review'].apply(lambda x: lda_model[dictionary.doc2bow(x.split())])

# Define a function to get the name of the most probable topic
def get_most_probable_topic(topic_distribution):
    most_probable_topic = max(topic_distribution, key=lambda x: x[1])
    topic_id, _ = most_probable_topic
    return f"Topic {topic_id + 1}"

# Create a new column with the names of the most probable topics
df['predicted_domain'] = df['lda_topics'].apply(get_most_probable_topic)

# Print the DataFrame with the new column
print(df[['domain', 'review', 'sentiment', 'predicted_domain']])


             domain                                             review  \
0      productivity  new version things entirely different aestheti...   
2      productivity  lot people use word standard  run problem use ...   
3      productivity  originally skeptical paying dollar 7 todo list...   
4      productivity  used go app note taking  year problems writing...   
5      productivity  use years  it s tagging capability unique  it ...   
...             ...                                                ...   
10868         games  new problem  game consistently mis moves piece...   
10896         games  amazing game love gets boring trying help thin...   
10903         games  among us pretty fun thing get disconnected reg...   
10921         games  game  literally filled memories  playing game ...   
10925         games  game good regards characters visuals controls ...   

       sentiment predicted_domain  
0              1          Topic 1  
2              0          Topic 1  
3  

In [39]:
from sklearn.metrics import accuracy_score


def map_topic_to_domain(topic_name):
    if '1' in topic_name:
        return 'productivity'
    elif '2' in topic_name:
        return 'social networking'
    elif '3' in topic_name:
        return 'games'
    else:
        return 'unknown'

# Map predicted topics to domain labels
df['predicted_domain'] = df['topic_names'].apply(map_topic_to_domain)
print(df)

# Calculate accuracy
accuracy = accuracy_score(df['domain'], df['predicted_domain'])

print(f"Accuracy: {accuracy * 100:.2f}%")


             domain                                             review  \
0      productivity  new version things entirely different aestheti...   
2      productivity  lot people use word standard  run problem use ...   
3      productivity  originally skeptical paying dollar 7 todo list...   
4      productivity  used go app note taking  year problems writing...   
5      productivity  use years  it s tagging capability unique  it ...   
...             ...                                                ...   
10868         games  new problem  game consistently mis moves piece...   
10896         games  amazing game love gets boring trying help thin...   
10903         games  among us pretty fun thing get disconnected reg...   
10921         games  game  literally filled memories  playing game ...   
10925         games  game good regards characters visuals controls ...   

       sentiment                                         lda_topics  \
0              1  [(0, 0.9473305), (1, 0

In [55]:
from pprint import pprint

# Test reviews
test_reviews = [
    "I really love using Evernote for my work. It helps me stay organized and productive.",
    "gaming has fun literally takes me back to childhood memories ",
    "I like to discuss with people on discord  ",
    "among gmail,discord,facebook,whatsapp best is discord",
    "google apps are so smooth ",
    "This is so much fun",
    "dog"
]

# Tokenize the test reviews
def preprocess_text(text):
    return text.lower().split()

tokenized_test_reviews = [preprocess_text(review) for review in test_reviews]

# Convert the test reviews to bag-of-words representation
bow_test_reviews = [dictionary.doc2bow(tokens) for tokens in tokenized_test_reviews]

# Infer topics for the test reviews
topics_test_reviews = [lda_model[bow] for bow in bow_test_reviews]

# Define a function to get the name of the most probable topic
def get_most_probable_topic(topic_distribution):
    most_probable_topic = max(topic_distribution, key=lambda x: x[1])
    topic_id, _ = most_probable_topic
    return f"Topic {topic_id + 1}"

# Get the names of the most probable topics for the test reviews
predicted_topics = [get_most_probable_topic(topics) for topics in topics_test_reviews]

# Print the predicted topics for the test reviews
# Print the predicted topics and associated keywords for the test reviews
#for i, (predicted_topic, topics) in enumerate(zip(predicted_topics, topics_test_reviews), start=1):
#   print(f"Predicted Topic for the test review {i}: {predicted_topic}")
#   print("Keywords:", [word for word, _ in lda_model.show_topic(topics[0][0])])  # Assuming lda_model is your trained LDA model
#   print()


def map_topic_to_domain(predicted_topic):
    if '1' in predicted_topic:
        return 'productivity'
    elif '2' in predicted_topic:
        return 'social networking'
    elif '3' in predicted_topic:
        return 'games'
    else:
        return 'unknown'

# Print the predicted topics and associated keywords for the test reviews
for i, (predicted_topic, topics) in enumerate(zip(predicted_topics, topics_test_reviews), start=1):
    domain_label = map_topic_to_domain(predicted_topic)
    print(f"Predicted Domain for the test review {i}: {domain_label} or {predicted_topic}")
   # print(f"Predicted Topic for the test review {i}: {predicted_topic}")
    #print("Keywords:", [word for word, _ in lda_model.show_topic(topics[0][0])])  # Assuming lda_model is your trained LDA model
    print()





Predicted Domain for the test review 1: productivity or Topic 1

Predicted Domain for the test review 2: games or Topic 3

Predicted Domain for the test review 3: social networking or Topic 2

Predicted Domain for the test review 4: social networking or Topic 2

Predicted Domain for the test review 5: productivity or Topic 1

Predicted Domain for the test review 6: games or Topic 3

Predicted Domain for the test review 7: games or Topic 3



In [56]:
df['domain'].value_counts()

productivity         2378
social networking    1974
games                 866
Name: domain, dtype: int64

In [57]:
df['predicted_domain'].value_counts()

Topic 1    2468
Topic 2    1922
Topic 3     828
Name: predicted_domain, dtype: int64

In [60]:
from sklearn.metrics import accuracy_score

# Function to map topic names to domain labels
def map_topic_to_domain(topic_name):
    if '1' in topic_name:
        return 'productivity'
    elif '2' in topic_name:
        return 'social networking'
    elif '3' in topic_name:
        return 'games'
    else:
        return 'unknown'


df['predicted_domain'] = df['topic_names'].apply(map_topic_to_domain)


# Calculating accuracy
accuracy = accuracy_score(df['domain'], df['predicted_domain'])

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 90.36%
